### 직방 서비스 원룸 데이터 수집
- 복잡한 데이터 수집방법
- 동이름 > 매물정보(데이터 프레임)

- 절차
    - 동이름 > 위도, 경도
    - 위도, 경도 > geohash code(지역 범위)
    - geohash > 매물 아이디
    - 매물 아이디 > 매물 정보

In [5]:
import pandas as pd
import requests

In [6]:
# 1. 동이름 > 위도, 경도

In [7]:
query = "망원동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={query}&serviceType=원룸"
response = requests.get(url)
datas = response.json()["items"][0]
lat, lng = datas["lat"], datas["lng"]
lat, lng

(37.556785583496094, 126.9013442993164)

In [2]:
# 2. 위도, 경도 > geohash

In [1]:
# !pip install geohash2

In [2]:
import geohash2

In [8]:
#precision : 영역 : 값이 커질수록 영역이 작아짐
code = geohash2.encode(lat, lng, precision = 5)
code

'wydjx'

In [3]:
#3. geohash > 매물 아이디

In [10]:
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={code}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
response = requests.get(url)
response

<Response [200]>

In [51]:
datas = response.json()['items']
datas[:3]

[{'section_type': None,
  'item_id': 30462357,
  'images_thumbnail': 'https://ic.zigbang.com/ic/items/30462357/1.jpg',
  'sales_type': '전세',
  'sales_title': '전세',
  'deposit': 8000,
  'rent': 0,
  'size_m2': 19.83,
  '공급면적': {'m2': 19.83, 'p': '6'},
  '전용면적': {'m2': 19.83, 'p': '6'},
  '계약면적': None,
  'room_type_title': None,
  'floor': '2',
  'floor_string': '2',
  'building_floor': '8',
  'title': '융자없는 풀옵션전세원룸 입주즉시가능 청소,도배완료',
  'is_first_movein': None,
  'room_type': '01',
  'address': '영등포구 당산동4가',
  'random_location': {'lat': 37.5295956445854, 'lng': 126.89760186342566},
  'is_zzim': False,
  'status': True,
  'service_type': '원룸',
  'tags': [],
  'address1': '서울시 영등포구 당산동4가',
  'address2': None,
  'address3': None,
  'manage_cost': '13',
  'reg_date': '2022-02-18T17:43:12+09:00',
  'is_new': False},
 {'section_type': None,
  'item_id': 30524930,
  'images_thumbnail': 'https://ic.zigbang.com/ic/items/30524930/1.jpg',
  'sales_type': '전세',
  'sales_title': '전세',
  'deposit': 7000

In [17]:
ids=[]
for data in datas:
    ids.append(data['item_id'])
ids[:5]

[30462357, 30524930, 30629925, 30681505, 30494154]

In [18]:
# list comprehention : 간단한 리스트 데이터를 만들때
ids = [data['item_id'] for data in datas]
ids[:3]

[30462357, 30524930, 30629925]

In [ ]:
# 4. 매물 아이디 > 매물 정보

In [19]:
url = "https://apis.zigbang.com/v2/items/list"
params = {"domain": "zigbang",
         "withCoalition": "true",
          "item_ids" : ids,
         }
response = requests.post(url, params)
response

<Response [200]>

In [28]:
items = response.json()["items"]
item_df = pd.DataFrame(items)
item_df.head()

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
0,None,30462357,https://ic.zigbang.com/ic/items/30462357/1.jpg,전세,전세,8000,0,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}",None,None,2,2,8,"융자없는 풀옵션전세원룸 입주즉시가능 청소,도배완료",None,01,영등포구 당산동4가,"{'lat': 37.5295956445854, 'lng': 126.897601863...",False,True,원룸,[],서울시 영등포구 당산동4가,None,None,13,2022-02-18T17:43:12+09:00,False
1,None,30524930,https://ic.zigbang.com/ic/items/30524930/1.jpg,전세,전세,7000,0,16.53,"{'m2': 16.53, 'p': '5'}","{'m2': 16.53, 'p': '5'}",None,None,7,7,8,💕놓칠수없는가격💕파격EVENT💕조절가능💕즉입가능💕,None,01,영등포구 당산동4가,"{'lat': 37.52908793122306, 'lng': 126.89758614...",False,True,원룸,[추천],서울시 영등포구 당산동4가,None,None,13,2022-02-07T16:03:16+09:00,False
2,None,30629925,https://ic.zigbang.com/ic/items/30629925/1.jpg,전세,전세,7000,0,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}",None,None,7,7,8,🧡더블역세권💥착한전세가💥밝은주변환경💥융자❌💛,None,01,영등포구 당산동4가,"{'lat': 37.52961538386757, 'lng': 126.89757715...",False,True,원룸,[추천],서울시 영등포구 당산동4가,None,None,12,2022-02-19T16:52:02+09:00,True
3,None,30681505,https://ic.zigbang.com/ic/items/30681505/1.jpg,전세,전세,7000,0,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}",None,None,7,7,8,★실제매물입니다★ 영등포구청역 융자없는 깔끔한 원룸입니다.,None,01,영등포구 당산동4가,"{'lat': 37.52958098130043, 'lng': 126.89759104...",False,True,원룸,[추천],서울시 영등포구 당산동4가,None,None,13,2022-02-17T15:58:52+09:00,False
4,None,30494154,https://ic.zigbang.com/ic/items/30494154/1.jpg,전세,전세,6000,0,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}",None,None,5,5,8,※양평역 원룸 전세※ #전세대출불가 #대로변 #가성비,None,01,영등포구 양평동3가,"{'lat': 37.529596643764094, 'lng': 126.8913187...",False,True,원룸,[],서울시 영등포구 양평동3가,None,None,12,2022-02-05T11:14:17+09:00,False


In [32]:
columns = ['item_id',
       'sales_title', 'deposit', 'rent',
        'floor', 'building_floor', 'title',
       'room_type', 'address', 
       'service_type', 'address1',
       'manage_cost', 'reg_date', 'is_new']


In [37]:
result_df = item_df[columns]
result_df = result_df[result_df["address"].str.contains(query)]
result_df = result_df.reset_index(drop=True)
result_df.head()

,item_id,sales_title,deposit,rent,floor,building_floor,title,room_type,address,service_type,address1,manage_cost,reg_date,is_new
0,30483493,전세,23000,0,6,6,"🦋합정역,신축1.5룸가성비짱,한강공원도보,보증보험OK",02,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-04T15:40:23+09:00,False
1,30608827,전세,23000,0,6,6,방1개 별도거실 주방 화장실 다용도실 분리형원룸 입니다,04,마포구 망원동,빌라,서울시 마포구 망원동,7,2022-02-12T16:27:27+09:00,False
2,30578522,월세,12000,10,3,5,💥LH 가능 깨끗한 방💥,01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-15T18:54:14+09:00,False
3,30726074,월세,12000,10,3,5,🎶 [LH 가능!] 🎶 [깔끔한 방!],01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-21T11:45:06+09:00,True
4,30340227,월세,1000,49,3,5,⭕망원역세권 7평원룸 내부깔끔⭕,01,마포구 망원동,원룸,서울시 마포구 망원동,3,2022-01-22T14:38:50+09:00,False


In [39]:
# 보증금 1억이하, 월세 100만원 이하
result_df = result_df[(result_df["deposit"] <=10000) & (result_df["rent"] <=100)]
result_df.head()

,item_id,sales_title,deposit,rent,floor,building_floor,title,room_type,address,service_type,address1,manage_cost,reg_date,is_new
4,30340227,월세,1000,49,3,5,⭕망원역세권 7평원룸 내부깔끔⭕,01,마포구 망원동,원룸,서울시 마포구 망원동,3,2022-01-22T14:38:50+09:00,False
5,30691402,월세,5000,90,4,4,🌈여성1인전용📢대출가능👑망원역6분🌈테라스💖,02,마포구 망원동,원룸,서울시 마포구 망원동,10,2022-02-18T12:12:34+09:00,False
6,30604487,전세,9300,0,2,5,망리단길 인접 풀옵션 원룸,01,마포구 망원동,원룸,서울시 마포구 망원동,3,2022-02-21T02:39:16+09:00,True
15,30687981,월세,1000,85,3,4,⭕망원역세권 남향투룸 내부깔끔⭕,04,마포구 망원동,빌라,서울시 마포구 망원동,5,2022-02-18T09:50:36+09:00,False
16,30488407,월세,500,43,옥탑방,3,🤩이넓은테라스가다내꺼🌞,02,마포구 망원동,원룸,서울시 마포구 망원동,2,2022-02-16T17:10:20+09:00,False


In [81]:
def oneroom(adress) :

    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={adress}&serviceType=원룸"
    response = requests.get(url)
    datas = response.json()["items"][0]
    lat, lng = datas["lat"], datas["lng"]
    
    code = geohash2.encode(lat, lng, precision = 5)
    
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={code}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
    response = requests.get(url)
    
    datas = response.json()["items"]
    
    ids = [data['item_id'] for data in datas]

    
    url = "https://apis.zigbang.com/v2/items/list"
    params = {"domain": "zigbang",
             "withCoalition": "true",
              "item_ids" : ids[:998],
             }
    response = requests.post(url, params)
    
    items = response.json()["items"]
    item_df = pd.DataFrame(items)
    
    columns = ['item_id',
       'sales_title', 'deposit', 'rent',
        'floor', 'building_floor', 'title',
       'room_type', 'address', 
       'service_type', 'address1',
       'manage_cost', 'reg_date', 'is_new']
    
    result_df = item_df[columns]
    result_df = result_df[result_df["address"].str.contains(adress)]
    result_df = result_df.reset_index(drop=True)
    
#     result_df = result_df[(result_df["deposit"] <=10000) & (result_df["rent"] <=100)]
    return result_df

In [80]:
oneroom("쌍용동")

,item_id,sales_title,deposit,rent,floor,building_floor,title,room_type,address,service_type,address1,manage_cost,reg_date,is_new
29,30090705,월세,300,31,2,3,✏✏좋은방✏✏크고 좋은 원룸입니다,01,천안시 서북구 쌍용동,원룸,충청남도 천안시 서북구 쌍용동,5,2022-02-15T11:58:01+09:00,False
30,30584820,월세,200,30,2,3,★남향+실매물+화장실 리모델링~,01,천안시 서북구 쌍용동,원룸,충청남도 천안시 서북구 쌍용동,5,2022-02-15T13:50:30+09:00,False
31,30659373,월세,300,31,2,4,실시간업데이트! 100%실매물!,02,천안시 서북구 쌍용동,원룸,충청남도 천안시 서북구 쌍용동,5,2022-02-16T11:28:36+09:00,False
32,30090974,월세,100,25,2,3,✏✏좋은방✏✏햇살잘드는 환한 방,02,천안시 서북구 쌍용동,원룸,충청남도 천안시 서북구 쌍용동,5,2022-02-15T11:58:21+09:00,False
33,30695488,월세,500,60,3,4,💯점가성비최고투룸입니다👊보증금탄력조절가능👊컨디션좋아요,04,천안시 서북구 쌍용동,빌라,충청남도 천안시 서북구 쌍용동,7,2022-02-18T15:55:28+09:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,30704330,월세,300,30,2,4,★남향+빌트인가구+주차장 좋아요,02,천안시 서북구 쌍용동,원룸,충청남도 천안시 서북구 쌍용동,5,2022-02-19T10:46:17+09:00,False
209,30731185,전세,5500,0,3,4,★분리형 투베이 전세★ 귀한 매물 선착순,02,천안시 서북구 쌍용동,원룸,충청남도 천안시 서북구 쌍용동,6,2022-02-21T15:32:32+09:00,True
210,30404449,월세,100,26,3,4,@분리형 원룸으로 깔끔한 풀옵션원룸입니다!,01,천안시 서북구 쌍용동,원룸,충청남도 천안시 서북구 쌍용동,5,2022-01-27T12:45:33+09:00,False
213,30531158,월세,2000,80,4,4,♣쓰리룸38평(상가주택)♣옵션없음,05,천안시 서북구 쌍용동,빌라,충청남도 천안시 서북구 쌍용동,0,2022-02-07T20:25:16+09:00,False
